# Artificial Neural Network
Constitutive law identification with Artificial Neural Network
(c) by Olivier Pantalé 2023

## Notes of version

# Initialization

In [ ]:
%run Common.ipynb
plotFigs = True
saveFigs = True

# Import Data from the HF5 file

Read Data from the HF5 file

In [ ]:
# Read the HF5 data file
DataFile = '3Cr2Mo'
#DataFile = 'JohnsonCook'
data = readH5(filename = DataFile + '.h5')

depsArray = getDepsparray(data)
tempArray = getTarray(data)
numberOfData = len(data)
print('depsArray',depsArray,'contains',depsArray.shape[0],'data')
print('tempArray',tempArray,'contains',tempArray.shape[0],'data')
print('total of',numberOfData,'data')

Plot the content of the data to see what it looks like

In [ ]:
if (plotFigs):
    plotDatas(data)

Assemble all data into one big chunk

In [ ]:
removeZero = False # Remove the first point where eps = 0

first = True
for d in data:
    subdata = np.ones_like(d[2])*np.array([d[0],d[1]])
    newdata = np.array([d[2][:,0], subdata[:,0], subdata[:,1], d[2][:,1]])
    if first :
        allData = newdata.T
        first = False
    else:
        allData = np.concatenate((allData, newdata.T))
print('Format of allData', allData.shape)
if (removeZero): 
    identData = allData[allData[:,0] != 0]
else:
    identData = allData
print('Format of identData', identData.shape)

Compute the value of $\dot\varepsilon_0$ as the minimum value of the array $\dot\varepsilon$

In [ ]:
deps0 = depsArray.min()
print('deps0=',deps0)

compute and replace $\dot\varepsilon$ by $\log(\frac{\dot\varepsilon}{\dot\varepsilon_0})$

In [ ]:
identData[:,1] = np.log(identData[:,1] / deps0)
print('Data deps',depsArray)
print('replaced by',np.unique(identData[:,1]))

Compute min and max of ranges

Computes ANNdata

In [ ]:
minData = identData.min(axis=0)
maxData = identData.max(axis=0)
rangeData = maxData - minData
annData = (identData - minData) / rangeData
print("Max error for normalized data is : %g" %(annData * rangeData + minData - identData).max())

Split Data into input and output

In [ ]:
annInput = annData[:,0:3]
annOutput = annData[:,3]
annInput.shape, annOutput.shape
nbInputs = 3
nbOutputs = 1

Define the structure of the ANN

In [ ]:
annTraining = True
models = []
convergences = []

activations = ['sigmoid','tanh','relu','softplus','swish','exponential']
shapes = [[17,9]]

loss = 'mean_squared_error'
optimizer = 'adam'

Build the ANN or load data from previous Save

In [ ]:
for activation in activations:
    for shape in shapes:
        # Initialize a sequential model
        model = Sequential()
        name = str(nbInputs)
        if type(shape) == list:
            first = True
            layNum = 0
            for layer in shape:
                if (first): 
                    model.add(Dense(layer, input_dim = nbInputs, activation = activation, name = 'hl-'+str(layNum)))
                else:
                    model.add(Dense(layer, activation = activation, name = 'hl-'+str(layNum)))
                first = False
                layNum += 1
                name += '-' + str(layer)
        else:
            model.add(Dense(layer, input_dim = nbInputs, activation = activation))
            name += '-' + str(layer)
        model.add(Dense(nbOutputs, name = 'output'))
        name += '-' + str(nbOutputs) + '-' + activation
        model._name = name
        models.append(model)
        model.compile(loss = loss, optimizer = optimizer)   
        convergences.append([name, np.array([])])

Print summary of the Artificial Neural Networks

In [ ]:
for model in models:
    model.summary()

In [ ]:
for convergence in convergences:
    print(convergence)

Train the new ANN

In [ ]:
epochs = 50
maxIterations = 20
savePeriod = 2
skipRecordIterations = 1
minLossReduction = 1e-8
windowLength = 61

In [ ]:
annTraining = False

In [ ]:
if annTraining:
    for model, convergence in zip (models, convergences):
        print("Start to train model :", model.name)
        iteration = 0
        lastMeanLoss = 1
        while(iteration < maxIterations):
            history = model.fit(annInput, annOutput, epochs = epochs, use_multiprocessing = True, verbose = 0, shuffle = True)
            loss = np.array(history.history['loss'])
            meanLoss = loss.mean()
            convergence[1] = np.append(convergence[1], loss)
            iteration += 1
            print('Iteration :', iteration, ': loss =', meanLoss, ': reduction =', lastMeanLoss - meanLoss)
            lastMeanLoss = meanLoss
            if (iteration % savePeriod):
                # Save Tensorflow model
                model.save(DataFile + '/' + model.name + '.h5')
                # Save convergence data
                writeArrayH5(DataFile + '/' + model.name + '-conv.h5', 'data', convergence[1])
        print('End of tranning phase with %g iterations and %g loops' %(iteration, iteration*epochs))
        # Save Tensorflow model
        model.save(DataFile + '/' + model.name + '.h5')
        # Save model manually
        writeAnnH5(DataFile + '/' + model.name + '-ANN.h5')
        # Save convergence data
        writeArrayH5(DataFile + '/' + model.name + '-conv.h5', 'data', convergence[1])
else:
    for model, convergence in zip (models, convergences):
        try:
            # Load Tensorflow model
            model.load_weights(DataFile + '/' + model.name + '.h5')
            # Load convergence data
            convergence[1] = readArrayH5(DataFile + '/' + model.name + '-conv.h5', 'data')
        except:
            continue

Show the convergence curves filtered using the savgol filter method

In [ ]:
for convergence in convergences:
    print('%s %.3f x 10-6' %(convergence[0], 1e6*np.mean(convergence[1][-100:])))

In [ ]:
for convergence in convergences:
    try:
        convergencef = savgol_filter(np.log10(convergence[1][epochs*skipRecordIterations:]), window_length = windowLength, polyorder = 2)
        plt.plot(convergencef, label = convergence[0], linewidth=2)
    except:
        continue
plt.ylim(None, -4)
plt.xlabel(r'ANN training epoch', fontsize = 16)
plt.ylabel(r'Training error : $\log_{10}\left(\text{E}_\text{MS}\right)$', fontsize = 16)
plt.title(r'Global convergence of the Artificial Neural Network models', fontsize = 16)
plt.legend()
if (saveFigs):
    plt.savefig('Figures/' + DataFile + '-convergence.svg')
plt.show()

Show prediction of models

In [ ]:
RMSEA = np.array([])
EAARA = np.array([])
for model in models:
    ARstress = (model.predict(annInput, verbose=0)*rangeData[3]+minData[3]).flatten()
    EAAR = np.sum(np.abs((identData[:,3] - ARstress)/(identData[:,3])))*100/ARstress.shape[0]
    RMSE = np.sqrt(np.sum((identData[:,3] - ARstress)**2)/ARstress.shape[0])
    RMSEA = np.append(RMSEA,RMSE)
    EAARA = np.append(EAARA,EAAR)
    print("Model %s" %(model.name))
    print('  RMSE = %.2f' %(RMSE)+' MPa')
    print("  EAAR = %.2f" %(EAAR) + ' %')

In [ ]:
SRMSE = np.argsort(RMSEA)
SEAAR = np.argsort(EAARA)
for i in SRMSE:
    print('model %s : RMSE=%.3f, ratio=%.3f' %(models[i].name,RMSEA[i],RMSEA[i]/RMSEA[SRMSE[0]]))
print('--------------')
for i in SEAAR:
    print('model %s : EAAR=%.3f, ratio=%.3f' %(models[i].name,EAARA[i],EAARA[i]/EAARA[SEAAR[0]]))

Function to plot results

In [ ]:
def plotResultsOfModel(model):
    Ts = getTarray(data)
    depsps = getDepsparray(data)
    ndepsp = depsps.shape[0]
    nT = Ts.shape[0]
    
    plt.figure(figsize = sbPlotSize(ndepsp))
    plt.rc('text', usetex = True)
    plt.subplots_adjust(hspace = 0.3)
    xs, ys = sbPlot(ndepsp)
    idx = 1
    for depsp in depsps:
        plt.subplot(xs, ys, idx)
        cl = 0
        for T in Ts:
            for d in data:
                if (d[0] == depsp and d[1] == T):
                    subdata = d[2]
                    plt.plot(subdata[10::25,0], subdata[10::25,1], label=r'$T=' + str(d[1]) + '^{\circ}C$', color=colors[cl], marker = 's', markersize = 5, linestyle = 'none')
                    inp = np.zeros((subdata.shape[0],3))
                    inp[:,0] = (subdata[:,0] - minData[0]) / rangeData[0]
                    inp[:,1] = (np.log(depsp / deps0) - minData[1]) / rangeData[1]
                    inp[:,2]  = (T - minData[2]) / rangeData[2]
                    plt.plot(subdata[:,0],model.predict(inp, verbose=0)*rangeData[3]+minData[3],colors[cl], linewidth = 2.5)
            cl += 1
        plt.xlabel(r'strain $\varepsilon$', fontsize = 16) # Labels the x axis
        plt.ylabel(r'flow stress $\sigma$ (MPa)', fontsize = 16) # Labels the y axis
        plt.title(r'strain rate $\dot{\varepsilon} = ' + str(depsp) + '$ s$^{-1}$', fontsize = 16) # Self explicit command
        plt.xlim(subdata[:,0].min(), subdata[:,0].max())
        #plt.ylim(bottom=0)
        idx += 1
        
    legendLines = []
    cl = 0
    for temp in list(Ts):
        legendLines.append((r'$T=$' + str(int(temp)) + r'$^{\circ}$C', {'color':colors[cl], 'linestyle':'-', 'linewidth':2.5, 'marker':'s'}))
        cl += 1
    
    if (ndepsp % 2):
        plt.legend([create_dummy_line(**l[1]) for l in legendLines], [l[0] for l in legendLines], 
               loc = 'upper center', fontsize = 12, ncols = 6, bbox_to_anchor = (1.0, -0.2), shadow = False)
    else:
        plt.legend([create_dummy_line(**l[1]) for l in legendLines], [l[0] for l in legendLines], 
               loc = 'upper center', fontsize = 12, ncols = 6, bbox_to_anchor = (0.0, -0.2), shadow = False)
        
    if (saveFigs):
        plt.savefig('Figures/' + DataFile + '-' + model.name + '.svg')
    
    plt.show()

Display the results

In [ ]:
if (plotFigs):
    for model in models:
        print(model.name)
        plotResultsOfModel(model)